In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Input, Conv2D, BatchNormalization
from keras.layers import MaxPool2D, MaxPooling2D, Reshape, Dropout, SeparableConv2D
from keras.models import Model
from keras.utils import to_categorical
import os

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline
from WavDataLoader import WavDataLoader
from WavDataGenerator import WavDataGenerator

Using TensorFlow backend.


In [7]:
labels = ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']
if os.name is 'nt':
    data_dir = r'C:\Development\kaggle\tensorflow-speech-recognition-challenge\data\train\audio'
else:
    data_dir = r'/home/shaur141/Development/kaggle/tensorflow-speech-recognition-challenge/data/train/audio'
    
wav_data_generator = WavDataGenerator(data_dir, labels, nx=128, ny=32, is_train=True, batch_size=32)
# wav_data_loader = WavDataLoader(data_dir, labels, nx=128, ny=32)


In [8]:
def build_model():
    inputs = Input(shape=(wav_data_generator.nx, wav_data_generator.ny,1))
#     x = Reshape((wav_data_loader.nx*wav_data_loader.ny,))(inputs)
#     x = BatchNormalization()(inputs)
    x = Conv2D(16,(3,3),strides=(1,1), activation='relu')(inputs)
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32,(3,3),strides=(2,2), activation='relu')(x)
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = SeparableConv2D(64,(3,3),strides=(2,2), activation='relu')(x) 
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)       
    x = Reshape((-1,))(x)
    x = Dense(512, activation='relu')(x)
#     x = Dense(128, activation='relu')(x)
    predictions = Dense(wav_data_generator.num_labels, activation='softmax')(x)    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model    

In [9]:
model = build_model()

In [10]:
# model.fit(x=wav_data_loader.X, y=to_categorical(wav_data_loader.y), validation_split=0.15, epochs=20)

model.fit_generator(wav_data_generator.generator(), 
                    steps_per_epoch=wav_data_generator.num_examples//wav_data_generator.batch_size)
                    

Epoch 1/1
  25/2008 [..............................] - ETA: 1441s - loss: 3.5056 - acc: 0.0437

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')